<a href="https://colab.research.google.com/github/Asma-Rafique/Asma-Rafique/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/MyDrive/Bert_model')  # Specify the path to the directory where the files are located


Mounted at /content/drive


In [3]:
!pip install transformers
import numpy as np
from sklearn.model_selection import train_test_split
import  config
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight
import  dataset
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import  engine
import  torch
from imblearn.under_sampling import RandomUnderSampler
from transformers import BertModel
import  pandas as pd
import  torch.nn as nn
from model import  BERT_Arch
from transformers import  AdamW

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.2 MB/s eta 0:00:00


In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# %%cu

def run():
    df=pd.read_csv('/content/drive/MyDrive/Bert_model/ActionItems_UrduMeetings_copied.txt', sep='\t',
                           names=["Sentences", "isActionItem"])
    label_encoder = preprocessing.LabelEncoder()
    df['isActionItem'] = label_encoder.fit_transform(df['isActionItem'])
    print(df['isActionItem'].value_counts())
    df = df.replace(to_replace='None', value=np.nan).dropna()
    train_text, val_text, train_labels, val_labels = train_test_split(df['Sentences'], df['isActionItem'],
                                                                        random_state=1,
                                                                        test_size=0.2,
                                                                        stratify=df['isActionItem'])

    print(df['isActionItem'].value_counts())
    #train_text = train_text['text']

    train_seq,train_mask,train_y,val_seq, val_mask, val_y=dataset.dataset(train_text, train_labels, val_text, val_labels)
    # define a batch size
    batch_size = 80

    # wrap tensors
    train_data = TensorDataset(train_seq, train_mask, train_y)

    # sampler for sampling the data during training
    train_sampler = RandomSampler(train_data)

    # dataLoader for train set
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # wrap tensors
    val_data = TensorDataset(val_seq, val_mask, val_y)

    # sampler for sampling the data during training
    val_sampler = SequentialSampler(val_data)

    # dataLoader for validation set
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    bert = BertModel.from_pretrained("bert-base-multilingual-uncased")
    for param in bert.parameters():
        param.requires_grad = False

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    # model= nn.DataParallel(model)
    model = BERT_Arch(bert)
    model = model.to(device)

    optimizer = AdamW(model.parameters(), lr=1e-3)

    # Load the model state_dict and optimizer state_dict if available
    # model.load_state_dict(torch.load('/content/drive/MyDrive/Bert_model/Multiligual_BERT_128_Urdu_verified_V.1.1.pt'))
    # optimizer.load_state_dict(torch.load('/content/drive/MyDrive/Bert_model/optimizer_state_dict.pt'))

    class_wts = compute_class_weight(class_weight="balanced", classes=np.unique(train_labels), y=train_labels)
    # convert class weights to tensor
    weights = torch.tensor(class_wts, dtype=torch.float)
    weights = weights.to(device)

    # loss function
    cross_entropy = nn.NLLLoss(weight=weights)
    epochs = 60

    # set initial loss to infinite
    best_valid_loss = float('inf')

    # empty lists to store training and validation loss of each epoch
    train_losses = []
    valid_losses = []

    # for each epoch
    for epoch in range(epochs):

        print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

        # train model
        train_loss, f1_train = engine.train(train_dataloader, model,device,optimizer,cross_entropy)

        # evaluate model
        valid_loss, f1_valid = engine.evaluate(model,val_dataloader,device,cross_entropy)

        # save the best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            # torch.save(model.state_dict(), '/content/drive/MyDrive/Bert_model/Multiligual_BERT_128_Urdu_verified_V.1.1.pt')
            torch.save(model.state_dict(), '/content/drive/MyDrive/Bert_model/Multiligual_BERT_Urdu_verified_V.1.1.pt')
            torch.save(optimizer.state_dict(), '/content/drive/MyDrive/Bert_model/Multiligual_BERToptimizer_V1.1.pt')

        # append training and validation loss
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        print(f'\nTraining Loss: {train_loss:.3f}')
        print(f'Validation Loss: {valid_loss:.3f}')
        print(f'\nTraining F1: {f1_train:.3f}')
        print(f'Validation F1: {f1_valid:.3f}')

if __name__ == "__main__":
    run()





1    26048
0     1229
Name: isActionItem, dtype: int64
1    26046
0     1229
Name: isActionItem, dtype: int64


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


cuda

 Epoch 1 / 60


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  Batch   100  of    273.
  Batch   200  of    273.

Evaluating...
  Batch    50  of     69.

Training Loss: 0.640
Validation Loss: 0.547

Training F1: 0.731
Validation F1: 0.798

 Epoch 2 / 60
  Batch   100  of    273.
  Batch   200  of    273.

Evaluating...
  Batch    50  of     69.

Training Loss: 0.598
Validation Loss: 0.507

Training F1: 0.787
Validation F1: 0.682

 Epoch 3 / 60
  Batch   100  of    273.
  Batch   200  of    273.

Evaluating...
  Batch    50  of     69.

Training Loss: 0.540
Validation Loss: 0.432

Training F1: 0.810
Validation F1: 0.828

 Epoch 4 / 60
  Batch   100  of    273.
  Batch   200  of    273.

Evaluating...
  Batch    50  of     69.

Training Loss: 0.503
Validation Loss: 0.430

Training F1: 0.829
Validation F1: 0.795

 Epoch 5 / 60
  Batch   100  of    273.
  Batch   200  of    273.

Evaluating...
  Batch    50  of     69.

Training Loss: 0.500
Validation Loss: 0.413

Training F1: 0.839
Validation F1: 0.835

 Epoch 6 / 60
  Batch   100  of    273.
  Ba

In [5]:
import torch
from sklearn.metrics import classification_report
from model import BERT_Arch
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel, BertTokenizer
#from flask.views import MethodView
import numpy as np
import  pandas as pd

class ActionItemsApI():

    def __init__(self):
        path = '/content/drive/MyDrive/Bert_model/Multiligual_BERT_Urdu_verified_V.1.1.pt'
        print(path)
        bert = BertModel.from_pretrained("bert-base-multilingual-uncased")
        self.MODEL = BERT_Arch(bert)
        self.DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.MODEL.load_state_dict(torch.load(path, map_location=self.DEVICE))
        self.MODEL.to(self.DEVICE)  # Changed this line
        self.TOKENIZER = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

    def sentence_prediction(self):
        df = pd.read_csv('/content/drive/MyDrive/Bert_model/test1.1.3.txt' , sep='\t',
                       names=["urdu_text", "is_sarcastic"])
        # label_encoder = LabelEncoder()

        label_encoder = preprocessing.LabelEncoder()
        df["is_sarcastic"] = label_encoder.fit_transform(df["is_sarcastic"])
        df = df.replace(to_replace='None', value=np.nan).dropna()
        # error come due to NAN values that split() can handle float so to remove error this line is used
        print("All okkk")
        test_text = df['urdu_text']
        test_labels= df['is_sarcastic']
        print(test_text)
        seq_len = [len(i.split()) for i in test_text]
        max_seq_len = max(seq_len)

        if max_seq_len > 128:
            max_seq_len = 128
        tokens_test = self.TOKENIZER.batch_encode_plus(
            test_text.tolist(),
            max_length=max_seq_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=False
        )

        test_seq = torch.tensor(tokens_test['input_ids'])
        test_mask = torch.tensor(tokens_test['attention_mask'])
        test_y = torch.tensor(test_labels.tolist())
        print('test',test_y)
        # get predictions for test data
        with torch.no_grad():
            preds = self.MODEL(test_seq.to(self.DEVICE), test_mask.to(self.DEVICE))
            print(preds)
            preds = preds.detach().cpu().numpy()

        # model's performance
        print(preds)
        preds = np.argmax(preds, axis=1)
        print(preds)
        print(classification_report(test_y, preds))


if __name__ == "__main__":
    api = ActionItemsApI()
    api.sentence_prediction()



/content/drive/MyDrive/Bert_model/Multiligual_BERT_Urdu_verified_V.1.1.pt
All okkk
0                                        وعلیکم السلام مس
1                                       وعلیکم السلام میم
2                                   ہاں جی کیا حال چال ہے
3                             اللہ کا شکر میم آپ کیسی ہیں
4                                  الحمد للہ مس ٹھیک ٹھاک
                              ...                        
1587    اور ٹھیک ہے اور سٹارٹ میں آپ ایک پیراگراف ایڈ ...
1588                             اور شارٹ میں اس کا پرپز 
1589                                               آلرائٹ
1590    ٹھِیک ہے پھر آپ اس کو فائنل کرو اور مجھے پھر ب...
1591    اس کے علاوہ آپ پراجیکٹ رپورٹ ریڈی کر کے دینا م...
Name: urdu_text, Length: 1591, dtype: object


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


test tensor([1, 1, 1,  ..., 1, 1, 1])
tensor([[-4.0021, -0.0184],
        [-4.1803, -0.0154],
        [-1.8962, -0.1627],
        ...,
        [-4.1861, -0.0153],
        [-0.0960, -2.3908],
        [-0.0235, -3.7616]], device='cuda:0')
[[-4.002124   -0.01844586]
 [-4.180291   -0.01541216]
 [-1.8962038  -0.16268069]
 ...
 [-4.1860766  -0.01532259]
 [-0.09602011 -2.3908236 ]
 [-0.02352028 -3.7616313 ]]
[1 1 1 ... 1 0 0]
              precision    recall  f1-score   support

           0       0.25      0.76      0.37       114
           1       0.98      0.82      0.89      1477

    accuracy                           0.82      1591
   macro avg       0.61      0.79      0.63      1591
weighted avg       0.93      0.82      0.85      1591

